In [ ]:
!pip install pyvis -q

In [105]:
%load_ext autoreload  
%autoreload 2
    
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import networkx as nx
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from scipy.stats import pearsonr
from dowhy.utils.plotting import plot
from dowhy import identify_effect_auto
from dowhy import identify_effect_id

from dowhy.causal_identifier.auto_identifier import AutoIdentifier, BackdoorAdjustment, EstimandType


from dowhy.causal_estimators.econml import Econml
from dowhy.causal_estimators.propensity_score_matching_estimator import PropensityScoreMatchingEstimator
from dowhy.causal_estimators.propensity_score_weighting_estimator import PropensityScoreWeightingEstimator
from dowhy.causal_estimators.propensity_score_stratification_estimator import PropensityScoreStratificationEstimator
from dowhy.causal_estimators.distance_matching_estimator import DistanceMatchingEstimator
from dowhy.causal_estimators.linear_regression_estimator import LinearRegressionEstimator 
from dowhy.causal_graph import CausalGraph

from dowhy.causal_refuters import (
    refute_bootstrap,
    refute_data_subset,
    refute_estimate,
    refute_dummy_outcome,
    refute_placebo_treatment,
    refute_random_common_cause,
    sensitivity_e_value,
    sensitivity_simulation
)

from dowhy.datasets import linear_dataset

from econml.dml import LinearDML
from econml.dr import LinearDRLearner 
from econml.metalearners import SLearner, TLearner, XLearner, DomainAdaptationLearner

from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, clone

import json

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', None)
np.random.seed(42)

c:\Users\eigho\anaconda4\envs\ampy_env_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [181]:
data = pd.read_csv('UCI_Credit_Card.csv/UCI_Credit_Card.csv', index_col=0)

### Preprocessing

In [182]:
data = data.rename(columns={'default.payment.next.month': 'def_pay', 
                        'PAY_0': 'PAY_1',
                         'LIMIT_BAL': 'CREDIT_AMOUNT'})

In [183]:
data = data[data['EDUCATION'] > 0]
data = data[data['MARRIAGE'] != 3]
data = data[data['SEX'] > 0]

In [24]:
data['SEX'] = data['SEX'] - 1 

In [22]:
data = data[data['EDUCATION'] != 6]

In [25]:
data = data.drop('EDUCATION', axis=1).merge(pd.get_dummies(data['EDUCATION'], prefix='EDU').astype('int'), on='ID')

In [26]:
data = data.rename(columns={'EDU_1':'EDU_GRADUATE_SCHOOL', 'EDU_2':'EDU_UNIVERSITY', 'EDU_3':'EDU_HIGH_SCHOOL', 'EDU_4':'EDU_OTHER', 'EDU_5':'EDU_UNKNOWN'})

In [27]:
data = data.drop('EDU_UNKNOWN', axis=1)

### BG generation

Cоздаем матрицу экспертных знаний (aka background knowledge):

In [29]:
bg_map = {i:node for i,node in enumerate(data.columns)}
bg_map_r = {node:i for i,node in enumerate(data.columns)}

bg = np.zeros((len(data.columns), len(data.columns))) - 1

Для простоты формирования background knowledge объединим переменные в группы:

In [30]:
demographic_fixed_vars = ['SEX', 'AGE']
demographic_other_vars = ['MARRIAGE']
education_vars = [col for col in data.columns if 'EDU' in col]
credit_desc_vars = ['CREDIT_AMOUNT']
target_variables = ['def_pay']

payment_vars_1 = [col for col in data.columns if '1' in col]
payment_vars_2 = [col for col in data.columns if '2' in col]
payment_vars_3 = [col for col in data.columns if '3' in col]
payment_vars_4 = [col for col in data.columns if '4' in col]
payment_vars_5 = [col for col in data.columns if '5' in col]
payment_vars_6 = [col for col in data.columns if '6' in col]

payments_vars = [payment_vars_1, payment_vars_2, payment_vars_3, payment_vars_4, payment_vars_5, payment_vars_6]

Избавляемся от направленных ребер в прошлое из будущего:

In [31]:
for i, payment_var in enumerate(payments_vars):
    for v_from in payment_var:
        for v_to in np.reshape(payments_vars[i+1:], -1):
            bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

for v_from in target_variables:
    for v_to in np.reshape(payments_vars, -1):
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

for v_from in target_variables:
    for v_to in demographic_fixed_vars + demographic_other_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

for v_from in np.reshape(payments_vars, -1):
    for v_to in credit_desc_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

Избавляемся от направленных ребер в демографические переменные:

In [32]:
for v_to in demographic_fixed_vars:
    for v_from in np.reshape(payments_vars, -1):
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0
    
    for v_from in credit_desc_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

    for v_from in education_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

    for v_from in demographic_other_vars + target_variables:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

Удаляем связи типа 'переменные баланса -> образование':

In [33]:
for v_from in np.reshape(payments_vars, -1):
    for v_to in education_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

for v_from in credit_desc_vars:
    for v_to in education_vars:
        bg[bg_map_r[v_from]][bg_map_r[v_to]] = 0

Скоррелированы, но не связаны (существует конфаундер вне датасета):

In [34]:
 bg[bg_map_r['AGE']][bg_map_r['SEX']] = 0
 bg[bg_map_r['SEX']][bg_map_r['AGE']] = 0

In [35]:
data.to_csv('uci_processed.csv')

### Causal discovery

In [42]:
alpha = 0.001
ci_test = 'fisherz'

g = pc(data.values, alpha=alpha, ci_test=ci_test)
bk = get_bk(g, bg)
g = pc(data.values, alpha=alpha, ci_test=ci_test, background_knowledge=bk)

g.to_nx_graph()

cg = g
g = nx.relabel_nodes(g.nx_graph, {prev_node:new_node for prev_node, new_node in zip(g.nx_graph.nodes, data.columns)})

Depth=13, working on node 26: 100%|██████████| 27/27 [00:00<00:00, 567.16it/s]


In [40]:
g.to_nx_graph()

cg = g
g = nx.relabel_nodes(g.nx_graph, {prev_node:new_node for prev_node, new_node in zip(g.nx_graph.nodes, data.columns)})

### Effect size estimation

In [195]:
data = data.astype('float')
df = data

In [197]:
for i in range(1, 7):
    df[f'PAY_{i}'] = (df[f'PAY_{i}'] > 0).astype('int')

In [1]:
tr = 'PAY_1'
eff = 'def_pay'

estimand = identify_effect_auto(
    graph=g,
    action_nodes=tr,
    outcome_nodes=eff,
    observed_nodes=list(g.nodes), 
    estimand_type=EstimandType.NONPARAMETRIC_ATE,
    backdoor_adjustment=BackdoorAdjustment.BACKDOOR_MAX
)

print(estimand)

NameError: name 'identify_effect_auto' is not defined

In [222]:
res = dict()

In [223]:
for est in [
    LinearRegressionEstimator, 
    DistanceMatchingEstimator,  
    PropensityScoreStratificationEstimator
]:
    estimator = est(
        identified_estimand=estimand,
    )
    estimator.fit(
        data=df
    )
    res[est.__name__] = estimator.estimate_effect(
        data=df,
        control_value=0,
        treatment_value=1,
        target_units="ate",
    )

In [215]:
for est in [
    PropensityScoreMatchingEstimator, 
    PropensityScoreWeightingEstimator,
]:
    estimator = est(
        identified_estimand=estimand,
    )
    estimator.fit(
        data=df
    )
    res[est.__name__] = estimator.estimate_effect(
        data=df,
        control_value=0,
        treatment_value=1,
        target_units="ate",
    )

#### SLearner

In [217]:
from utils import RegressionWrapper, PropensityWrapper

In [218]:
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=SLearner(        
        overall_model=RandomForestRegressor(random_state=42, max_depth=6)
    )
)
estimator.fit(
    data=df
)
res['SLearner'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### TLearner

In [219]:
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=TLearner(
        models=RandomForestRegressor(random_state=42, max_depth=6),
    ),
)
estimator.fit(
    data=df
)

res['TLearner'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### XLearner

In [205]:
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=XLearner(
        models=RandomForestRegressor(random_state=42, max_depth=6),
        cate_models=RandomForestRegressor(random_state=42, max_depth=6),
        propensity_model=RandomForestClassifier(random_state=42, max_depth=6)
    ),
)
estimator.fit(
    data=df
)

res['XLearner'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### DomainAdaptationLearner

In [206]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=DomainAdaptationLearner(
        models=RandomForestRegressor(random_state=42, max_depth=6),
        final_models=RandomForestRegressor(random_state=42, max_depth=6),
        propensity_model=PropensityWrapper(CalibratedClassifierCV(RandomForestClassifier(random_state=42, max_depth=6), cv=cv))
    ),
)
estimator.fit(
    data=df
)

res['DALearner'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### DML

In [207]:
cg = CausalGraph(
    treatment_name=tr, 
    outcome_name=eff,
    graph=g
)
cg.get_effect_modifiers([tr], [eff])

In [208]:
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=LinearDML(
        model_y=RandomForestRegressor(random_state=42, max_depth=6),
        model_t=RandomForestClassifier(random_state=42, max_depth=6),
        featurizer=PolynomialFeatures(degree=3, include_bias=True),
        discrete_treatment=True
    ),
)
estimator.fit(
    data=df,
    effect_modifier_names=cg.get_effect_modifiers([tr], [eff]),
)

res['DML'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### DRLearner

In [209]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
estimator = Econml(
    identified_estimand=estimand,
    econml_estimator=LinearDRLearner(
        model_regression=RandomForestRegressor(random_state=42, max_depth=6),
        model_propensity=PropensityWrapper(CalibratedClassifierCV(RandomForestClassifier(random_state=42, max_depth=6), cv=cv)),
        featurizer=PolynomialFeatures(degree=3, include_bias=True)
    ),
)
estimator.fit(
    data=df,
    effect_modifier_names=cg.get_effect_modifiers([tr], [eff]),
)

res['DR'] = estimator.estimate_effect(
    data=df,
    control_value=0,
    treatment_value=1,
    target_units="ate",
)

#### Результаты

In [210]:
for k, v in res.items():
    print(k, v.value)

LinearRegressionEstimator 0.3130968851730703
SLearner 0.362933556158329
TLearner 0.37004214239834277
XLearner 0.3632530488896904
DistanceMatchingEstimator 0.3161514344469541
PropensityScoreStratificationEstimator 0.3729361119889048
PropensityScoreMatchingEstimator 0.2641674813741024
PropensityScoreWeightingEstimator 0.21904466872029793
DALearner 0.36984147548607926
DML 0.2702556779444086
DR 0.36453275415245173


In [141]:
estimand.set_identifier_method('backdoor')

In [142]:

br = refute_bootstrap(
    # запустить refute_bootstrap
    # https://www.pywhy.org/dowhy/main/dowhy.causal_refuters.html#dowhy.causal_refuters.bootstrap_refuter.refute_bootstrap   
    #  Указать: num_simulations=99
    df, estimand, res['LinearRegressionEstimator'], show_progress_bar=True, num_simulations=99
)
print(br)

Refuting Estimates: 100%|██████████| 99/99 [00:05<00:00, 18.29it/s]

Refute: Bootstrap Sample Dataset
Estimated effect:0.03958119115518702
New effect:0.03872324600942011
p value:0.434820805887834



In [143]:
sr = refute_data_subset(df, estimand, res['LinearRegressionEstimator'], num_simulations=99, show_progress_bar=True)
print(sr)

Refuting Estimates: 100%|██████████| 99/99 [00:04<00:00, 23.32it/s]

Refute: Use a subset of data
Estimated effect:0.03958119115518702
New effect:0.039985149865737944
p value:0.43694870053540663



In [147]:
do_n_r = refute_dummy_outcome(
    df, 
    estimand, 
    res['LinearRegressionEstimator'],
    num_simulations=99, 
    outcome_name=eff,
    treatment_name=[tr], 
    required_variables=True,
    show_progress_bar=True
)
print(do_n_r[0])

Refuting Estimates: 100%|██████████| 99/99 [00:04<00:00, 20.13it/s]

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.0009407067152714752
p value:0.4702253544602577



In [148]:
do_p_r = refute_dummy_outcome(
    df, 
    estimand, 
    res['LinearRegressionEstimator'],
    num_simulations=99, 
    outcome_name=eff,
    treatment_name=[tr], 
    transformation_list=[("permute", {"permute_fraction":1})],
    required_variables=True,
    show_progress_bar=True
)
print(do_p_r[0])

Refuting Estimates: 100%|██████████| 99/99 [00:04<00:00, 19.84it/s]

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-5.294590470451734e-05
p value:0.496569337728166



In [150]:
ptr = refute_placebo_treatment(
    #запустить
    df, estimand, res['LinearRegressionEstimator'], treatment_names=[tr], num_simulations=99, show_progress_bar=True)
print(ptr)

Refuting Estimates: 100%|██████████| 99/99 [00:04<00:00, 20.07it/s]

Refute: Use a Placebo Treatment
Estimated effect:0.03958119115518702
New effect:0.0
p value:nan



In [151]:
ccr = refute_random_common_cause(df, estimand, res['LinearRegressionEstimator'], num_simulations=99, show_progress_bar=True)
print(ccr)

Refuting Estimates: 100%|██████████| 99/99 [00:05<00:00, 19.29it/s]

Refute: Add a random common cause
Estimated effect:0.03958119115518702
New effect:0.039582559810278126
p value:0.48171175634399804

